In [0]:
# Command to remove tables from databricks storage.
#dbutils.fs.rm('/FileStore/shared_uploads/ddink002@ucr.edu/dataset_repub_before_election_geo_tagged.csv',recurse=True)

In [0]:
from pyspark.sql import SparkSession

from pyspark import SparkContext
#sc=SparkContext(appName="Data_App")    
    
spark = SparkSession.builder \
        .appName("Big_Data_Project_App").getOrCreate()

In [0]:
df1 = spark.read.format("csv").option("header","true").load("/FileStore/tables/dataset_repub_before_election_raw.csv")

In [0]:
df1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Tweet Id: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- User_Location: string (nullable = true)
 |-- coordinate_longitude: string (nullable = true)
 |-- coordinate_latitude: string (nullable = true)
 |-- place_Fullname: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- countryCode: string (nullable = true)



In [0]:
# get no of rows in dataframe
rows = df1.count()
print(f"DataFrame Rows count : {rows}")

DataFrame Rows count : 9368108


In [0]:
# Drop Unnecessary Columns
df1 = df1.drop("_c0")
df1.printSchema()

root
 |-- Datetime: string (nullable = true)
 |-- Tweet Id: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- User_Location: string (nullable = true)
 |-- coordinate_longitude: string (nullable = true)
 |-- coordinate_latitude: string (nullable = true)
 |-- place_Fullname: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- countryCode: string (nullable = true)



In [0]:
# Remove data with User_Location Null.
df1 = df1.na.drop(subset=["User_Location"])
df1.show()

+--------------------+-------------------+--------------------+---------------+--------------------+--------------------+-------------------+--------------+----------+-------------+-----------+
|            Datetime|           Tweet Id|                Text|       Username|       User_Location|coordinate_longitude|coordinate_latitude|place_Fullname|place_name|place_country|countryCode|
+--------------------+-------------------+--------------------+---------------+--------------------+--------------------+-------------------+--------------+----------+-------------+-----------+
|2022-11-07 23:59:...|1589769667190882304|@chrislhayes Mayb...|   EricEElliott|          Austin, TX|                null|               null|          null|      null|         null|       null|
|2022-11-07 23:59:...|1589769666846945281|@ProudElephantUS ...|GeorgeB20642540|          Olathe, KS|                null|               null|          null|      null|         null|       null|
|2022-11-07 23:59:...|15897696

In [0]:
# get no of rows in dataframe
rows = df1.count()
print(f"DataFrame Rows count : {rows}")

DataFrame Rows count : 3090761


In [0]:
# Adding new column State_Name to store final state name. 
from pyspark.sql.functions import lit
df1 = df1.withColumn("State_Name", lit(''))
df1.printSchema()

root
 |-- Datetime: string (nullable = true)
 |-- Tweet Id: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- User_Location: string (nullable = true)
 |-- coordinate_longitude: string (nullable = true)
 |-- coordinate_latitude: string (nullable = true)
 |-- place_Fullname: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- State_Name: string (nullable = false)



In [0]:
df1 = df1.withColumn("State_Code", lit(''))
df1.printSchema()

root
 |-- Datetime: string (nullable = true)
 |-- Tweet Id: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- User_Location: string (nullable = true)
 |-- coordinate_longitude: string (nullable = true)
 |-- coordinate_latitude: string (nullable = true)
 |-- place_Fullname: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- State_Name: string (nullable = false)
 |-- State_Code: string (nullable = false)



In [0]:
df1.printSchema()

root
 |-- Datetime: string (nullable = true)
 |-- Tweet Id: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- User_Location: string (nullable = true)
 |-- coordinate_longitude: string (nullable = true)
 |-- coordinate_latitude: string (nullable = true)
 |-- place_Fullname: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- State_Name: string (nullable = false)
 |-- State_Code: string (nullable = false)



In [0]:
# Get abbreviation from state_name 
stateNames = {
    'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE',
    'District of Columbia': 'DC','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS',
    'Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS',
    'Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY',
    'North Carolina': 'NC','North Dakota': 'ND','Northern Mariana Islands':'MP','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Palau': 'PW',
    'Pennsylvania': 'PA','Puerto Rico': 'PR','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT',
    'Vermont': 'VT','Virgin Islands': 'VI','Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY',
}
# Get list of state names mapped from their abbreviation to name
stateCodes = dict(map(reversed, stateNames.items()))

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# import string library function 
import string

In [0]:
# Function to do keyword search User_Location for USA States. 
cnt = 0
@f.udf(returnType='STRUCT<`State_Name`: string, `State_Code`: string, `country`: string, `code`: string>')
def do_geocode(address1):
  cnt = 0
  if address1 is None:
        return {'State_Name': None, 'State_Code': None, 'country': None, 'code': None}
    
  print(address1)
  tok = address1
  state =''
  for i in tok.split(', '):
    if stateCodes.get(i) is not None:
      state = stateCodes.get(i)
  if state == '':
    for i in tok.split(', '):
      if i in stateNames:
        state = i
  if state=='':
    for i in tok.split(' '):
      if stateCodes.get(i) is not None:
        state = stateCodes.get(i)
    if state == '':
      for i in tok.split(' '):
        if i in stateNames:
          state = i
  print(state)
  print(cnt)
  cnt=cnt+1
  country1 = 'United States'
  code1 = 'US'
  if state != '':
      return {'State_Name': state, 'State_Code': stateNames[state], 'country': country1, 'code': code1}
  else :    
      return {'State_Name': None, 'State_Code': None, 'country': None, 'code': None}       

In [0]:
raw_cols = df1.columns

In [0]:
# Apply fucntion to the dataframe
GeocodedLocation_df = df1.select(raw_cols)\
                            .withColumn("location", \
                            do_geocode(df1["User_Location"]))


In [0]:
GeocodedLocation_df.printSchema()

root
 |-- Datetime: string (nullable = true)
 |-- Tweet Id: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- User_Location: string (nullable = true)
 |-- coordinate_longitude: string (nullable = true)
 |-- coordinate_latitude: string (nullable = true)
 |-- place_Fullname: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- State_Name: string (nullable = false)
 |-- State_Code: string (nullable = false)
 |-- location: struct (nullable = true)
 |    |-- State_Name: string (nullable = true)
 |    |-- State_Code: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- code: string (nullable = true)



In [0]:
# Copy the data from location column to other column based on respective information. 
df2=GeocodedLocation_df.withColumn("State_Name", GeocodedLocation_df.location.State_Name)

df2=df2.withColumn("State_Code", df2.location.State_Code)

df2=df2.withColumn("countrycode", df2.location.code)

df2=df2.withColumn("place_country", df2.location.country)

df2.printSchema()

root
 |-- Datetime: string (nullable = true)
 |-- Tweet Id: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- User_Location: string (nullable = true)
 |-- coordinate_longitude: string (nullable = true)
 |-- coordinate_latitude: string (nullable = true)
 |-- place_Fullname: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- State_Name: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- State_Name: string (nullable = true)
 |    |-- State_Code: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- code: string (nullable = true)



In [0]:
# Drop Location column
df2 = df2.drop(df2.location)
df2.printSchema()

root
 |-- Datetime: string (nullable = true)
 |-- Tweet Id: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- User_Location: string (nullable = true)
 |-- coordinate_longitude: string (nullable = true)
 |-- coordinate_latitude: string (nullable = true)
 |-- place_Fullname: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- State_Name: string (nullable = true)
 |-- State_Code: string (nullable = true)



In [0]:
df2.show()

+--------------------+-------------------+--------------------+---------------+--------------------+--------------------+-------------------+--------------+----------+-------------+-----------+--------------------+----------+
|            Datetime|           Tweet Id|                Text|       Username|       User_Location|coordinate_longitude|coordinate_latitude|place_Fullname|place_name|place_country|countrycode|          State_Name|State_Code|
+--------------------+-------------------+--------------------+---------------+--------------------+--------------------+-------------------+--------------+----------+-------------+-----------+--------------------+----------+
|2022-11-07 23:59:...|1589769667190882304|@chrislhayes Mayb...|   EricEElliott|          Austin, TX|                null|               null|          null|      null|United States|         US|               Texas|        TX|
|2022-11-07 23:59:...|1589769666846945281|@ProudElephantUS ...|GeorgeB20642540|          Olathe,

In [0]:
# Reorder schema
df2 = df2.select(df2.columns[0],df2.columns[1],df2.columns[2],df2.columns[3],df2.columns[4],df2.columns[5],df2.columns[6],df2.columns[7],df2.columns[8],df2.columns[11],df2.columns[12],df2.columns[9],df2.columns[10])

df2.printSchema()

root
 |-- Datetime: string (nullable = true)
 |-- Tweet Id: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- User_Location: string (nullable = true)
 |-- coordinate_longitude: string (nullable = true)
 |-- coordinate_latitude: string (nullable = true)
 |-- place_Fullname: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- State_Name: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- countrycode: string (nullable = true)



In [0]:
# Remove rows with State_Name as null or None.
df_final = df2.na.drop(subset=["State_Name"])
df_final.show()

+--------------------+-------------------+--------------------+---------------+--------------------+--------------------+-------------------+--------------+----------+--------------------+----------+-------------+-----------+
|            Datetime|           Tweet Id|                Text|       Username|       User_Location|coordinate_longitude|coordinate_latitude|place_Fullname|place_name|          State_Name|State_Code|place_country|countrycode|
+--------------------+-------------------+--------------------+---------------+--------------------+--------------------+-------------------+--------------+----------+--------------------+----------+-------------+-----------+
|2022-11-07 23:59:...|1589769667190882304|@chrislhayes Mayb...|   EricEElliott|          Austin, TX|                null|               null|          null|      null|               Texas|        TX|United States|         US|
|2022-11-07 23:59:...|1589769666846945281|@ProudElephantUS ...|GeorgeB20642540|          Olathe,

In [0]:
# get no of rows in dataframe
rows = df_final.count()
print(f"DataFrame Rows count : {rows}")

DataFrame Rows count : 1715771


In [0]:
# Select only US tweets
df_final = df_final.filter(df_final.countrycode == 'US')
df_final.show()

+--------------------+-------------------+--------------------+---------------+--------------------+--------------------+-------------------+--------------+----------+--------------------+----------+-------------+-----------+
|            Datetime|           Tweet Id|                Text|       Username|       User_Location|coordinate_longitude|coordinate_latitude|place_Fullname|place_name|          State_Name|State_Code|place_country|countrycode|
+--------------------+-------------------+--------------------+---------------+--------------------+--------------------+-------------------+--------------+----------+--------------------+----------+-------------+-----------+
|2022-11-07 23:59:...|1589769667190882304|@chrislhayes Mayb...|   EricEElliott|          Austin, TX|                null|               null|          null|      null|               Texas|        TX|United States|         US|
|2022-11-07 23:59:...|1589769666846945281|@ProudElephantUS ...|GeorgeB20642540|          Olathe,

In [0]:
# get no of rows in dataframe
rows = df_final.count()
print(f"DataFrame Rows count : {rows}")

DataFrame Rows count : 1715771


In [0]:
# Display and download the dataframe as csv
display(df_final)

Datetime Tweet Id Text Username User_Location coordinate_longitude coordinate_latitude place_Fullname place_name State_Name State_Code place_country countrycode 2022-11-07 23:59:59+00:00 1589769667190882304 @chrislhayes Maybe we can just pretend it didn’t happen like Republicans do January 6th… EricEElliott Austin, TX null null null null Texas TX United States US 2022-11-07 23:59:59+00:00 1589769666846945281 @ProudElephantUS GOAT GOP!!!! GeorgeB20642540 Olathe, KS null null null null Kansas KS United States US 2022-11-07 23:59:59+00:00 1589769665018204160 Yes please listen to him and keep our democracy alive by #VoteBlueTomorrow because the @GOP don’t want you to vote and they don’t want you to have the same rights they do. They have voted no on everything including #Veterans healthcare! Please #VoteBlueToSaveDemocracy #Roevember MICHELELYNLAWR1 San Marcos, TX null null null null Texas TX United States US 2022-11-07 23:59:57+00:00 1589769659372666880 @elonmusk Democrats want to mitigate climate change so humans can live on Earth, close the unsustainable wealth gap, clean air & water, women's right to choose, racial equality, and ending mass shootings. #GOP= tax cuts for rich, gun violence, voter restriction, health care for rich. estate_nova Arlington, Virginia null null null null Virginia VA United States US 2022-11-07 23:59:57+00:00 1589769656025624577 Republicans and social conservatives may feel enormous urgency to write their cultural priorities into law before liberal-leaning Millennials and Generation Z become the electorate’s dominant force later this decade, @RonBrownstein reports: https://t.co/PLcF65UdfP northassoc Northampton, MA null null null null Massachusetts MA United States US 2022-11-07 23:59:55+00:00 1589769648459116546 @Jim_Jordan Republicans cannot offer anything besides lies, end of Social Security, Medicare, VA benefits, free and fair elections. Republicans have no solutions on any of our challenges. https://t.co/I40gWDuEU0 ingridkirschba3 North Barrington, IL null null null null Illinois IL United States US 2022-11-07 23:59:54+00:00 1589769642859704320 "@stooge_report @MoonStarCyn If Republicans take the House and Senate, they'd better call ALL these doctors, especially Fraudci, in to testify & get to the bottom of the fraud that was committed against the American people! Every last 1 of them shld be tried for TREASON! No ""Covid Forgiveness!"" Disgusting!🤬" LinKrenicki Florida, USA null null null null Florida FL United States US 2022-11-07 23:59:52+00:00 1589769637402923010 @w_terrence Republicans are the cheats and sore losers davitrusty2 Michigan, USA null null null null Michigan MI United States US 2022-11-07 23:59:51+00:00 1589769633737080834 @sharon_bovee @ericareport We’ll the Keystone XL might have been online come this spring if it wasn’t halted. Because Big oil was pouting the raised prices and have the largest profits in their history. KXL had nothing to do with prices. Republicans have a hard time with this fact. recreate2000 Rapid City, SD null null null null South Dakota SD United States US 2022-11-07 23:59:50+00:00 1589769630024749057 @psgsr160 @JuanMoreNews @BillClinton @RepCuellar You'd trust this vile person over Republicans? How yummy is that kool-aid? LeahSSchaefer Texas, USA null null null null Texas TX United States US 2022-11-07 23:59:49+00:00 1589769623544778754 #Republicans Think. Vote Republican RaySr1946 Illinois, USA null null null null Illinois IL United States US 2022-11-07 23:59:49+00:00 1589769623423311872 The rally was about 20 minutes long, wrapping up in time for folks to get to the Herschel Walker rally literally down the road one mile. Why Walker wasnt at the GOP unity rally with every other Republican statewide candidate? 🤷‍♀️ Grace_Segers Washington, DC null null null null District of Columbia DC United States US 2022-11-07 23:59:47+00:00 1589769617198964736 Also their cross tabs from entire track challenge idea of big working class Latino move to GOP. @NALEO has D

In [0]:
df4.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("/FileStore/tables/dataset_repub_before_election_filtered_with_State.csv")

In [0]:
spark.stop()